## **Predicciones**

In [1]:
# Cargar librerías
import os
import pandas as pd
from tensorflow.keras.models import load_model
import joblib


2026-01-18 00:38:56.095242: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 00:38:56.104684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768718336.114776   19776 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768718336.118066   19776 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768718336.126967   19776 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

from functions.prediction import predict_raw_version_a, predict_raw_version_b, predict_raw_ensamble

In [3]:
# Rutas de los archivos 
path_mlp_r = "../models/mlp_residual_model.keras"
path_mlp_r_assets = "../models/mlp_r_assets.joblib"
path_xgb_log = "../models/xgboost_log.pkl"
path_ensamble_log = "../models/ensamble_model_log.pkl"

# Cargar MLP residual (Versión A)
if (os.path.exists(path_mlp_r)) & (os.path.exists(path_mlp_r_assets)):
    model_mlp_r = load_model(path_mlp_r)
    bundle_mlp_r = joblib.load(path_mlp_r_assets)
    print("MLP residual cargado")
else:
    print("No se encontró MLP residual")

# Cargar XGBoost Log (Versión B)
if os.path.exists(path_xgb_log):
    bundle_xgb_log = joblib.load(path_xgb_log)
    model_xgb = bundle_xgb_log["model"]
    print("XGBoost Log")
else:
    print("No se encontró XGBoost Log")

# Cargar Ensamble Log (Versión Ensamble)
if os.path.exists(path_ensamble_log):
    bundle_ensamble_log = joblib.load(path_ensamble_log)
    print("Ensamble Log cargado")
else:
    print("No se encontró Ensamble Log")

2026-01-18 00:38:57.437633: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2026-01-18 00:38:57.437656: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:167] env: CUDA_VISIBLE_DEVICES="-1"
2026-01-18 00:38:57.437661: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2026-01-18 00:38:57.437665: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2026-01-18 00:38:57.437670: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: andres-alvarez
2026-01-18 00:38:57.437672: I external/local_xla/xla/stream_executor/cud

MLP residual cargado
XGBoost Log
Ensamble Log cargado


In [4]:
print(bundle_xgb_log["features"])

['power_kw', 'fuel_consumption_l_100km', 'mileage_in_km', 'age', 'model_encoded', 'brand_encoded', 'fuel_type_encoded', 'color_encoded', 'transmission_type_encoded']


In [5]:
# Datos de prueba (precio real: 1300 €)

test_case_1 = pd.DataFrame([{
    "brand": "alfa-romeo",
    "model": "Alfa Romeo GTV",
    "color": "red",
    "registration_date": "10/1995",
    "year": 1995,
    "power_kw": 148,
    "power_ps": 201,
    "transmission_type": "Manual",
    "fuel_type": "Petrol",
    "fuel_consumption_l_100km": "10.9 l/100 km",  
    "fuel_consumption_g_km": "260 g/km",
    "mileage_in_km": 160500.0,
    "offer_description": "2.0 V6 TB"
}])

test_case_2 = pd.DataFrame([{
    "brand": "alfa-romeo ",       # Espacio extra al final
    "model": "Alfa Romeo GTV",
    "color": "RED",               # Mayúsculas inconsistentes
    "registration_date": "1995-10", # Formato de fecha diferente
    "year": "1995",               # String en lugar de entero
    "power_kw": "148,0",          # Coma decimal y string
    "power_ps": 201,
    "transmission_type": "manual", # Minúsculas
    "fuel_type": "Petrol",
    "fuel_consumption_l_100km": "10,9 l/100 km", # Coma en lugar de punto
    "fuel_consumption_g_km": "260", # Falta la unidad "g/km"
    "mileage_in_km": "160500",   # String con punto separador de miles
    "offer_description": "2.0 V6 TB"
}])

test_case_3 = pd.DataFrame([{
    "brand": "alfa-romeo",
    "model": "Alfa Romeo GTV",
    # "color": FALTANTE
    "registration_date": "10/1995",
    "year": 1995,
    "power_kw": 148,
    # "power_ps": FALTANTE
    "transmission_type": "Manual",
    "fuel_type": "Petrol",
    # "fuel_consumption_l_100km": FALTANTE
    "fuel_consumption_g_km": "260 g/km",
    "mileage_in_km": 160500.0,
    "offer_description": "2.0 V6 TB"
}])

test_case_4 = pd.DataFrame([{
    "brand": "alfa-romeo",
    "model": "Alfa Romeo GTV"
    # Todo lo demás falta
}])


In [6]:
def test_prediction(predictions_list, name_model, name_test, df, predict_function, bundle, model=None):
    if model:
        price = predict_function(df, bundle, model)
    else:
        price = predict_function(df, bundle)

    predictions = {
        "Modelo": name_model,
        "Test": name_test,
        "Predicción (€)": f"{price[0]:,.2f}"
    }

    predictions_list.append(predictions)
    return predictions_list

# Lista para almacenar las predicciones
predictions_list = []

# Predicciones de MLP
predictions_list = test_prediction(
    predictions_list, "MLP residual", "test_case_1", test_case_1, 
    predict_raw_version_a, bundle_mlp_r, model_mlp_r
)
predictions_list = test_prediction(
    predictions_list, "MLP residual", "test_case_2", test_case_2, 
    predict_raw_version_a, bundle_mlp_r, model_mlp_r
)
predictions_list = test_prediction(
    predictions_list, "MLP residual", "test_case_3", test_case_3, 
    predict_raw_version_a, bundle_mlp_r, model_mlp_r
)
predictions_list = test_prediction(
    predictions_list, "MLP residual", "test_case_4", test_case_4, 
    predict_raw_version_a, bundle_mlp_r, model_mlp_r
)

# Predicciones de XGBoost Log
predictions_list = test_prediction(
    predictions_list, "XGBoost Log", "test_case_1", test_case_1,
    predict_raw_version_b, bundle_xgb_log, model_xgb
)
predictions_list = test_prediction(
    predictions_list, "XGBoost Log", "test_case_2", test_case_2,
    predict_raw_version_b, bundle_xgb_log, model_xgb
)
predictions_list = test_prediction(
    predictions_list, "XGBoost Log", "test_case_3", test_case_3,
    predict_raw_version_b, bundle_xgb_log, model_xgb
)
predictions_list = test_prediction(
    predictions_list, "XGBoost Log", "test_case_4", test_case_4,
    predict_raw_version_b, bundle_xgb_log, model_xgb
)

# Predicciones de Ensamble Log
predictions_list = test_prediction(
    predictions_list, "Ensamble Log", "test_case_1", test_case_1,
    predict_raw_ensamble, bundle_ensamble_log
)
predictions_list = test_prediction(
    predictions_list, "Ensamble Log", "test_case_2", test_case_2,
    predict_raw_ensamble, bundle_ensamble_log
)
predictions_list = test_prediction(
    predictions_list, "Ensamble Log", "test_case_3", test_case_3,
    predict_raw_ensamble, bundle_ensamble_log
)
predictions_list = test_prediction(
    predictions_list, "Ensamble Log", "test_case_4", test_case_4,
    predict_raw_ensamble, bundle_ensamble_log
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [7]:
df_predictions = pd.DataFrame(predictions_list)

df_predictions = df_predictions.sort_values(by="Test", ascending=True).reset_index(drop=True)

df_predictions

,Modelo,Test,Predicción (€)
0,MLP residual,test_case_1,"5,867.59"
1,XGBoost Log,test_case_1,"2,843.33"
2,Ensamble Log,test_case_1,"3,807.24"
3,MLP residual,test_case_2,"5,867.59"
4,XGBoost Log,test_case_2,"2,843.33"
5,Ensamble Log,test_case_2,"3,807.24"
6,MLP residual,test_case_3,"4,418.61"
7,XGBoost Log,test_case_3,"2,713.70"
8,Ensamble Log,test_case_3,"3,694.83"
9,MLP residual,test_case_4,"20,686.20"
